# Context Engineering Dashboard — Basic Usage

This notebook demonstrates how to visualize LLM context windows using the `context-engineering-dashboard` package.

In [ ]:
# Install (if needed)
# !pip install context-engineering-dashboard[all]

## 1. Manual Trace Construction

Build a trace manually to understand the data structure.

In [ ]:
from context_engineering_dashboard import (
    ContextTrace,
    ContextComponent,
    ComponentType,
    ContextWindow,
)

# Create components manually
components = [
    ContextComponent(
        id="sys_1",
        type=ComponentType.SYSTEM_PROMPT,
        content="You are a helpful assistant specializing in ChromaDB documentation.",
        token_count=2000,
    ),
    ContextComponent(
        id="rag_1",
        type=ComponentType.RAG_DOCUMENT,
        content="# Getting Started with ChromaDB\n\nChromaDB is an open-source embedding database...",
        token_count=8000,
        metadata={"chroma_score": 0.92, "source": "docs/getting-started.md"},
    ),
    ContextComponent(
        id="rag_2",
        type=ComponentType.RAG_DOCUMENT,
        content="# Collections\n\nA collection is the main container for documents...",
        token_count=3000,
        metadata={"chroma_score": 0.87, "source": "docs/collections.md"},
    ),
    ContextComponent(
        id="history_1",
        type=ComponentType.CHAT_HISTORY,
        content="Previous turn: User asked about installation...",
        token_count=1000,
    ),
    ContextComponent(
        id="user_1",
        type=ComponentType.USER_MESSAGE,
        content="How do I create a collection in Chroma?",
        token_count=350,
    ),
]

# Create trace
trace = ContextTrace(
    context_limit=128_000,
    components=components,
    total_tokens=sum(c.token_count for c in components),
)

print(f"Total tokens: {trace.total_tokens:,}")
print(f"Utilization: {trace.utilization:.1f}%")

## 2. Visualize with ContextWindow

In [ ]:
# VIEW mode (default)
ctx = ContextWindow(trace=trace, context_limit=128_000)
ctx.display()

In [ ]:
# EXPLORE mode (double-click components to see details)
ctx = ContextWindow(trace=trace, context_limit=128_000, mode="explore")
ctx.display()

## 3. Trace with OpenAI

Automatically capture context from OpenAI calls.

In [ ]:
from context_engineering_dashboard import trace_openai
from openai import OpenAI

client = OpenAI()

with trace_openai() as tracer:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful coding assistant."},
            {"role": "user", "content": "Explain what context engineering is in 2 sentences."},
        ],
        temperature=0.7,
    )

# Visualize
ctx = ContextWindow(trace=tracer.result)
ctx.display()

## 4. Trace with Chroma RAG

Full RAG workflow with Chroma retrieval tracing.

In [ ]:
import chromadb
from context_engineering_dashboard import trace_chroma, ContextWindow

# Setup Chroma
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection("docs")

# Add some documents
collection.add(
    ids=["doc_1", "doc_2", "doc_3", "doc_4", "doc_5"],
    documents=[
        "ChromaDB is an open-source embedding database designed for AI applications.",
        "To create a collection, use client.create_collection('name').",
        "Embeddings can be generated using OpenAI, Cohere, or local models.",
        "The query method returns the most similar documents based on cosine similarity.",
        "Metadata filtering allows you to narrow down results using key-value pairs.",
    ],
    metadatas=[
        {"source": "overview.md"},
        {"source": "collections.md"},
        {"source": "embeddings.md"},
        {"source": "querying.md"},
        {"source": "filtering.md"},
    ],
)

In [ ]:
# Wrap collection for tracing
traced = trace_chroma(collection)

# Query (captures all returned docs)
results = traced.query(
    query_texts=["How do I create a collection?"],
    n_results=5,
)

# Mark which docs we select for context (e.g., top 2)
traced.mark_selected(["doc_2", "doc_1"])

# Add other context components
traced.add_system_prompt("You are a helpful assistant for ChromaDB.")
traced.add_user_message("How do I create a collection?")

# Get trace and visualize
trace = traced.get_trace(context_limit=128_000)
ctx = ContextWindow(trace=trace, mode="explore", show_available_pool=True)
ctx.display()

## 5. Serialization

Save and load traces for later analysis.

In [ ]:
# Save to JSON
trace.to_json("my_trace.json")

# Load from JSON
loaded_trace = ContextTrace.from_json("my_trace.json")

# Verify
print(f"Components: {len(loaded_trace.components)}")
print(f"Total tokens: {loaded_trace.total_tokens:,}")

## 6. Compare Before/After with Sankey Diff

In [ ]:
from context_engineering_dashboard import ContextDiff

# Create a "before" trace with more content
before_components = [
    ContextComponent(id="sys", type=ComponentType.SYSTEM_PROMPT, content="...", token_count=4000),
    ContextComponent(id="hist", type=ComponentType.CHAT_HISTORY, content="...", token_count=20000),
    ContextComponent(id="rag", type=ComponentType.RAG_DOCUMENT, content="...", token_count=15000),
    ContextComponent(id="tool", type=ComponentType.TOOL, content="...", token_count=1000),
]
before = ContextTrace(
    context_limit=128_000,
    components=before_components,
    total_tokens=40000,
)

# Create an "after" trace (compacted)
after_components = [
    ContextComponent(id="sys", type=ComponentType.SYSTEM_PROMPT, content="...", token_count=4000),
    ContextComponent(id="hist", type=ComponentType.CHAT_HISTORY, content="...", token_count=8000),
    ContextComponent(id="rag", type=ComponentType.RAG_DOCUMENT, content="...", token_count=10000),
    ContextComponent(id="tool", type=ComponentType.TOOL, content="...", token_count=1000),
]
after = ContextTrace(
    context_limit=128_000,
    components=after_components,
    total_tokens=23000,
)

# Visualize diff
diff = ContextDiff(
    before=before,
    after=after,
    before_label="Original",
    after_label="After /compact",
)
diff.sankey()

In [ ]:
# Print text summary
diff.summary()